### Processamento de Linguagem
O Multinomial Naive Bayes é muito utilizado para processamento de linguagem, porém antes de aplicá-lo necessitamos fixar alguns conceitos.

O computador trabalha muito bem com processamento de números, mas não com processamento de strings/textos. Para que o algoritmo funcione de maneira correta, precisamos transformar as palavras em números. O método mais simples de executar essa tranformação é criar um vetor/matriz com a contagem de cada palavra presente em nosso corpus.

Processo:  
Corpus > Tokenization > Bag of words

**O que é um Corpus?**  
Corpus é uma coleção de documentos. No exemplo abaixo temos um corpus com dois documentos (docA e docB):

In [1]:
# Exemplo de Corpus
docA = "O gato esta sentado no sofa"
docB = "O cachorro esta sentado no sofa"
docC = "O passarinho esta sentado no poleiro"

corpus = [docA, docB, docC]
corpus

['O gato esta sentado no sofa',
 'O cachorro esta sentado no sofa',
 'O passarinho esta sentado no poleiro']

**Tokenization**  
Para poder trabalhar com os documentos, necessitamos seperar cada palavra para que o algoritmo possa interpretá-las, tranformando-as em uma **Bag of words** (saco de palavras). Essa transformação é conhecida como tokenization.

In [2]:
# Exemplo de tokenization
docA1 = docA.split(" ")
docB1 = docB.split(" ")

# Exibição dos documentos tokenizados
docA1, docB1

(['O', 'gato', 'esta', 'sentado', 'no', 'sofa'],
 ['O', 'cachorro', 'esta', 'sentado', 'no', 'sofa'])

**Bag of Words**  
A partir dos documentos tokenizados, podemos tranformá-los em um saco de palavras. O método mais simples é criando um vetor com a contagem de cada palavra.

In [3]:
# Unindo os dois documentos em palavras únicas
wordset = set(docA1).union(set(docB1))
wordset

{'O', 'cachorro', 'esta', 'gato', 'no', 'sentado', 'sofa'}

In [4]:
# Criando um dicionário vazio para a contagem de palavras
wordDictA = dict.fromkeys(wordset, 0)
wordDictB = dict.fromkeys(wordset, 0)

wordDictA

{'sentado': 0, 'sofa': 0, 'cachorro': 0, 'esta': 0, 'no': 0, 'O': 0, 'gato': 0}

In [5]:
# Calculando a frequência de cada palavra dos documentos A e B
for i in docA1:
    wordDictA[i]+=1
    
for i in docB1:
    wordDictB[i]+=1

In [6]:
# Transformando numa matriz de frequência
import pandas as pd
pd.DataFrame([wordDictA, wordDictB])

,O,cachorro,esta,gato,no,sentado,sofa
0,1,0,1,1,1,1,1
1,1,1,1,0,1,1,1


Um problema em contarmos cada palavra em nossa bag of words é que palavras muito utilizadas e de pouca importância podem acabar inflenciando o algoritmo. No exemplo acima, palavras como "o", "está" e "no" aparecem mais que as demais e pouco acrescentam à analise.

O padrão das línguas naturais seguem a [Lei de Zipf](https://pt.wikipedia.org/wiki/Lei_de_Zipf). Trata-se de uma lei de potências (contrapartida à distribuição gaussiana) sobre a distribuição de valores de acordo com o número de ordem numa lista. Nessa lista, o membro n teria uma relação de valor com o 1º da lista na proporçao 1/n. Isto significa que o segundo elemento se repetirá aproximadamente com uma frequência que é metade da do primeiro, e o terceiro elemento com uma frequência de 1/3 e assim sucessivamente.

<img src="figuras/Graphique_Zipf_pour_Ulysses.png" width="500"/>


Devido à esse padrão, caso fôssemos analizar um texto grande, talvez não conseguiríamos extrair algum insight valioso, pois preposições e conjuções provavelmente apareceriam mais do que substantivos e adjetivos. Uma abordagem melhor é utilizar o [Tf-Idf](https://pt.wikipedia.org/wiki/Tf%E2%80%93idf), explicado adiante.

### Módulo feature_extraction.text
O módulo ``feature_extraction.text`` do Scikit-Learn possui alguns utilitários para construir vetores de recursos numéricos a partir de documentos de texto. Se olharmos dentro do módulo, encontraremos três classes diferentes que podem transformar o texto em recursos numéricos: ``CountVectorizer``, ``HashingVectorizer`` e ``TfidfVectorizer``. A diferença entre eles reside nos cálculos que executam para obter os recursos numéricos.

**CountVectorizer**  

Basicamente cria um dicionário de palavras a partir do corpus(coleção de documentos). Em seguida, cada instância é convertida em um vetor de recursos numéricos, em que cada elemento será a contagem do número de vezes que uma determinada palavra aparece no documento. 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
CVect = CountVectorizer()

In [8]:
vector1 = CVect.fit_transform(corpus)

In [9]:
# Para verificarmos a contagem de palavras, usamos o .toarray()
vector1.toarray()

array([[0, 1, 1, 1, 0, 0, 1, 1],
       [1, 1, 0, 1, 0, 0, 1, 1],
       [0, 1, 0, 1, 1, 1, 1, 0]], dtype=int64)

In [10]:
# Para verificar a que palavra cada coluna pertence, usamos .get_feature_names()
CVect.get_feature_names()

['cachorro', 'esta', 'gato', 'no', 'passarinho', 'poleiro', 'sentado', 'sofa']

In [11]:
# Montando em um DataFrame para melhor vizualização
pd.DataFrame(vector1.toarray(), columns=CVect.get_feature_names())

,cachorro,esta,gato,no,passarinho,poleiro,sentado,sofa
0,0,1,1,1,0,0,1,1
1,1,1,0,1,0,0,1,1
2,0,1,0,1,1,1,1,0


In [12]:
# Se quisermos olhar as palavras dentro de um documento em específico, utilizamos o .inverse_transform
a = vector1.toarray()
CVect.inverse_transform(a[2])

[array(['esta', 'no', 'passarinho', 'poleiro', 'sentado'], dtype='<U10')]

**TfidfVectorizer**  
Funciona como o CountVectorizer, mas com um cálculo mais avançado chamado Term Frequency Inverse Document Frequency (TF-IDF). Usa uma estatística para medir a importância de uma palavra em um documento ou corpus. Intuitivamente, procura palavras mais frequentes no documento atual, comparadas com a frequência em todo o corpus de documentos. Podemos ver isso como uma maneira de normalizar os resultados e evitar palavras que são muito freqüentes e, portanto, inúteis para caracterizar as instâncias.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
TfidfVect = TfidfVectorizer()

In [14]:
# Convertendo
vector2 = TfidfVect.fit_transform(corpus)

In [15]:
# Montando em um DataFrame para melhor vizualização
pd.DataFrame(vector2.toarray(), columns=TfidfVect.get_feature_names())

,cachorro,esta,gato,no,passarinho,poleiro,sentado,sofa
0,0.000000,0.364544,0.617227,0.364544,0.000000,0.000000,0.364544,0.469417
1,0.617227,0.364544,0.000000,0.364544,0.000000,0.000000,0.364544,0.469417
2,0.000000,0.338381,0.000000,0.338381,0.572929,0.572929,0.338381,0.000000


Como pode ser visto no quadro acima, as palavras 'cachorro', 'gato', 'passarinho' e 'poleiro' tiveram um peso maior que 'sofá', que por sua vez teve um peso maior que 'está', 'no' e 'sentado'. O que o algoritmo fez foi dar um peso maior às palavras que se repetiram menos. Perceba tabmém que na linha 2 as palavras em comum com as demais linhas tiveram um peso diferente, como por exemplo a palavra 'sentado'. Isso ocorreu porque a frase dessa linha possui mais palavras diferentes do que as linhas 0 e 1 (poleiro em vez de sofá).

**Stop words**
Como dito anteriormente o texto analisado pode apresentar alta incidência de palavras de pouca relevância (p.ex. artigos, preposições e conjuções) e por mais que atenuemos o peso através da ponderação do TF_IDF, ainda sim pode acabar prejudicando a análise. Uma maneira de diminuir esse problema é utilizando *Stop Words* para retirar essas palavras.  

*Stop words* (ou palavras de parada – tradução livre) são palavras que podem ser consideradas irrelevantes para o conjunto de resultados a ser exibido em uma busca realizada em uma search engine. Exemplos: as, e, os, de, para, com, sem, foi.

Para utilizar stop words no vectorizer, basta configurar o parâmetro ``stop_words``:

    TfidfVectorizer(stop_words='english')

Podemos intuir que temos um conjunto de stop words para cada idioma. Se o nosso texto estiver em inglês, basta utilizar o exemplo acima, porém se estiver em portugues teremos que baixar a stop word através do pacote ``nltk``.

In [16]:
import nltk

nltk.download('stopwords')  # baixando as stop words

portugues = nltk.corpus.stopwords.words('portuguese')  # escolhendo o idioma português

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rildo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# Visualizando o início da lista
pd.DataFrame(portugues).head(10)

,0
0,de
1,a
2,o
3,que
4,e
5,do
6,da
7,em
8,um
9,para


In [18]:
# Configurando o parâmetro stop_words no TfidfVectorizer
TfidfVect1 = TfidfVectorizer(stop_words=portugues)

In [19]:
# Convertendo
vector3 = TfidfVect1.fit_transform(corpus)

In [20]:
# Exemplo com stop words
pd.DataFrame(vector3.toarray(), columns=TfidfVect1.get_feature_names())

,cachorro,gato,passarinho,poleiro,sentado,sofa
0,0.000000,0.720333,0.000000,0.000000,0.425441,0.547832
1,0.720333,0.000000,0.000000,0.000000,0.425441,0.547832
2,0.000000,0.000000,0.652491,0.652491,0.385372,0.000000


Podemos ver que ao utilizar stop words, as palavras 'esta' e 'no' foram retiradas do vetor e a importância das demais foi aumentada. Com isso o algoritmo pode trabalhar de maneira mais correta.

In [21]:
# Exemplo sem stop words
pd.DataFrame(vector2.toarray(), columns=TfidfVect.get_feature_names())

,cachorro,esta,gato,no,passarinho,poleiro,sentado,sofa
0,0.000000,0.364544,0.617227,0.364544,0.000000,0.000000,0.364544,0.469417
1,0.617227,0.364544,0.000000,0.364544,0.000000,0.000000,0.364544,0.469417
2,0.000000,0.338381,0.000000,0.338381,0.572929,0.572929,0.338381,0.000000
